In [ ]:
import uuid
import os
import requests
import pandas as pd
import geopandas as gpd
from dotenv import load_dotenv
from shapely.geometry import Point
from time import sleep

### Add UUIDs

In [ ]:
aldi_df = pd.read_csv("../data/scraped_stores.csv")
aldi_df = aldi_df.iloc[:, 1:]
aldi_df["uuid"] = [str(uuid.uuid4()) for _ in range(aldi_df.shape[0])]

### Geocode Stores

In [ ]:
load_dotenv("../.env")
google_api_key = os.environ.get("GOOGLE_API_KEY")

base_url = "https://maps.googleapis.com/maps/api/geocode/json?address="

def geocode(address: str) -> Point:
    sleep(0.02)
    print(f"Geocoding {address}")
    res = requests.get(f"{base_url}ALDI, {address}&key={google_api_key}")

    if res.status_code != 200:
        print(f"FAILED: {res.status_code}")
        return None
    
    if res.json().get("status") == "ZERO_RESULTS":
        print(f"FAILED: No results")
        return None
    
    location = res.json().get("results")[0].get("geometry").get("location")
    lat = location.get("lat")
    lon = location.get("lng")
    return Point(lon, lat)

aldi_df["geometry"] = aldi_df["full_address"].apply(geocode)
aldi_gdf = gpd.GeoDataFrame(aldi_df, crs="EPSG:4326")

In [ ]:
manual_geocode = {
    251: Point(-1.763692473483625, 53.81636898310299),
    345: Point(-4.2137573022227075, 55.854817001581196),
    389: Point(-0.7487276312703152, 51.30766641207592),
    400: Point(-0.688799660073178, 52.03520827497847),
    425: Point(-0.5431430024338115, 51.32362945134225),
    559: Point(-1.8969025023805457, 52.49677080787523),
    842: Point(-4.169406786883136, 55.779735347026595),
    867: Point(-3.4688902175386027, 56.41950541807793),
    962: Point(-1.5190461735490193, 52.402724341328124)
}

for key in manual_geocode.keys():
    aldi_gdf.loc[key, "geometry"] = manual_geocode[key]